# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from imblearn import over_sampling, under_sampling
from imblearn.pipeline import Pipeline

# Load data

In [ ]:
# Đường dẫn file 
train_transaction_data_file = "../input/ieee-fraud-detection/train_transaction.csv"
test_transaction_data_file = "../input/ieee-fraud-detection/test_transaction.csv"
train_identity_data_file = "../input/ieee-fraud-detection/train_identity.csv"
test_identity_data_file = "../input/ieee-fraud-detection/test_identity.csv"
sample_submission_file = "../input/ieee-fraud-detection/sample_submission.csv"

In [ ]:
# Đọc data
train_transaction_data = pd.read_csv(train_transaction_data_file)
train_identity_data = pd.read_csv(train_identity_data_file)
test_transaction_data = pd.read_csv(test_transaction_data_file)
test_identity_data = pd.read_csv(test_identity_data_file)
sample_submission = pd.read_csv(sample_submission_file)
del train_transaction_data_file, test_transaction_data_file, train_identity_data_file, test_identity_data_file, sample_submission_file

# Giảm kích cỡ data

In [ ]:
# hàm giảm kích cỡ
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# Giảm kích cỡ data
train_transaction_data = reduce_mem_usage(train_transaction_data)
train_identity_data = reduce_mem_usage(train_identity_data)
test_transaction_data = reduce_mem_usage(test_transaction_data)
test_identity_data = reduce_mem_usage(test_identity_data)

# Đọc và Ghép nối data

In [ ]:
# Ghép nối transaction_data và indentity_data
train_data = train_transaction_data.merge(train_identity_data, how="left", on="TransactionID")
test_data = test_transaction_data.merge(test_identity_data, how="left", on="TransactionID")

del train_transaction_data, train_identity_data, test_transaction_data, test_identity_data

In [ ]:
# Đổi tên cột các cột id cho giống với train data
test_data.rename({'id-01':'id_01','id-02':'id_02','id-03':'id_03','id-04':'id_04','id-05':'id_05','id-06':'id_06','id-07':'id_07','id-08':'id_08','id-09':'id_09','id-10':'id_10','id-11':'id_11','id-12':'id_12','id-13':'id_13','id-14':'id_14','id-15':'id_15','id-16':'id_16','id-17':'id_17','id-18':'id_18','id-19':'id_19','id-20':'id_20','id-21':'id_21','id-22':'id_22','id-23':'id_23','id-24':'id_24','id-25':'id_25','id-26':'id_26','id-27':'id_27','id-28':'id_28','id-29':'id_29','id-30':'id_30','id-31':'id_31', 'id-32':'id_32', 'id-33':'id_33', 'id-34':'id_34', 'id-35':'id_35', 'id-36':'id_36', 'id-37':'id_37', 'id-38':'id_38'}, axis=1, inplace=True)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

# Feature engineering

**Email domain**

Có nhiều trường email của cùng domain nhưng nó tồn tại ở nhiều dạng khác nhau. Thông tin chi tiết ở:
https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654

Việc quy về một 1 dạng domain sẽ giảm được nhiều rất là nhiều.

In [ ]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
for c in ['P_emaildomain', 'R_emaildomain']:
    train_data[c + '_bin'] = train_data[c].map(emails)
    test_data[c + '_bin'] = test_data[c].map(emails)
    
    train_data[c + '_suffix'] = train_data[c].map(lambda x: str(x).split('.')[-1])
    test_data[c + '_suffix'] = test_data[c].map(lambda x: str(x).split('.')[-1])
    
    train_data[c + '_suffix'] = train_data[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test_data[c + '_suffix'] = test_data[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [ ]:
del emails, us_emails

**Xử lý cột id_31**

In [ ]:
# Quy hết các thông tin về browser thành các thông tin về browser ở một số lượng nhất định
train_data.loc[train_data['id_31'].str.contains('chrome', na=False), 'id_31'] = 'Chrome'
train_data.loc[train_data['id_31'].str.contains('firefox', na=False), 'id_31'] = 'Firefox'
train_data.loc[train_data['id_31'].str.contains('safari', na=False), 'id_31'] = 'Safari'
train_data.loc[train_data['id_31'].str.contains('edge', na=False), 'id_31'] = 'Edge'
train_data.loc[train_data['id_31'].str.contains('ie', na=False), 'id_31'] = 'IE'
train_data.loc[train_data['id_31'].str.contains('samsung', na=False), 'id_31'] = 'Samsung'
train_data.loc[train_data['id_31'].str.contains('opera', na=False), 'id_31'] = 'Opera'
train_data['id_31'].fillna("NAN", inplace=True)
train_data.loc[train_data.id_31.isin(train_data.id_31.value_counts()[train_data.id_31.value_counts() < 200].index), 'id_31'] = "Others"

**V features**

V features (V1-V339) như ở notebook trước ta đã biết là có rất nhiều chiều. Ở đây ta có thể áp dụng PCA để giảm số chiều của V xuống

In [ ]:
def PCA_change(df, cols, n_components, prefix='PCA_', rand_seed=4):
    pca = PCA(n_components=n_components, random_state=rand_seed)

    principalComponents = pca.fit_transform(df[cols])

    principalDf = pd.DataFrame(principalComponents)

    df.drop(cols, axis=1, inplace=True)

    principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

    df = pd.concat([df, principalDf], axis=1)
    
    return df

In [ ]:
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA


for col in train_data.columns[55:394]:
    train_data[col] = train_data[col].fillna((train_data[col].min() - 2))
    train_data[col] = (minmax_scale(train_data[col], feature_range=(0,1)))

for col in test_data.columns[54:393]:
    test_data[col] = test_data[col].fillna((test_data[col].min() - 2))
    test_data[col] = (minmax_scale(test_data[col], feature_range=(0,1)))
    
train_data_PCA = PCA_change(train_data, train_data.columns[55:394], prefix='PCA_V_', n_components=50)
test_data_PCA = PCA_change(test_data, test_data.columns[54:393], prefix='PCA_V_', n_components=50)

# Chuẩn bị data cho modeling

In [ ]:
# Lọc ra các cột có giá trị null lớn hơn 95%
many_null_cols = [col for col in train_data.columns if train_data[col].isnull().sum() / train_data.shape[0] > 0.95]
many_null_cols_test = [col for col in test_data.columns if test_data[col].isnull().sum() / test_data.shape[0] > 0.95]

In [ ]:
print(len(many_null_cols))
print(len(many_null_cols_test))

In [ ]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test))
len(cols_to_drop)

In [ ]:
train_data = train_data.drop(cols_to_drop, axis=1)
test_data = test_data.drop(cols_to_drop, axis=1)

train_data_PCA = train_data_PCA.drop(cols_to_drop, axis=1)
test_data_PCA = test_data_PCA.drop(cols_to_drop, axis=1)

In [ ]:
del many_null_cols, many_null_cols_test, cols_to_drop

In [ ]:
# Tách label với các trường data khác
Y = train_data['isFraud'].copy()
X = train_data.drop('isFraud', axis=1)

In [ ]:
# thông tin train data sau khi giảm kích cỡ
del train_data

In [ ]:
# Chọn các cột categorical
categorical_cols = [cname for cname in X.columns if
                   X[cname].dtype == "object"]

# Chọn các cột numerical
numerical_cols = [cname for cname in X.columns if 
                  X[cname].dtype in ['int8', 'int16', 'int32', 'float16', 'float32']]

In [ ]:
# Tiền xử lý cho numerical data
numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')), ('scale', StandardScaler())])


# Tiền xử lý cho categorical data
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                                           ('onehot', OneHotEncoder(dtype=np.int8, handle_unknown='ignore'))])

# Ghép 2 tiền xử lý cho numerical và categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
# Chia tập train và validation
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.3, train_size=0.7, random_state=0)
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

del X, Y

Như đã nói ở notebook trước, đó là sự mất cân bằng class trong datasets, điều này có thể làm model dễ bị overfitting khi nó cố gắng fit dữ liệu và ngộ nhận tất cả giao dịch đều không gian lận. Có một số cách để giải quyết vấn đề này:
1. Oversample giúp tăng số lượng ở class ít dữ liệu hơn, dữ liệu được cân bằng hơn. Nhưng nó có thể làm tăng khả năng xảy ra overfitting, vì nó tạo ra các bản sao chính xác example của class ít số lượng hơn.
2. Undersample thì có tác dụng làm giảm số lượng data trong class nhiều hơn để giúp đạt được phân phối mong muốn giữa 2 class nhưng hạn chế của nó là có thể nó sẽ xóa đi những data mang nhiều thông tin quan trọng và không có cách nào để giữ lại trong dataset những thông tin hữu ích của class có số lượng nhiều hơn.
3. Resample là đơn giản là sự kết hợp của oversample và undersample để có được tỉ lệ 2 class như mong muốn.

Ở đây trong bài toán này, ta vẫn đề cao việc giữ cho kích cỡ mẫu lớn và dùng oversamling nhẹ (tránh vấn đề đã nêu trên) để cải thiện hiệu suất model

In [ ]:
# Oversampling data
oversampling = over_sampling.RandomOverSampler(sampling_strategy=0.1)
# undersampling = under_sampling.RandomUnderSampler(sampling_strategy=0.05)
steps = [('o', oversampling)]
pipeline = Pipeline(steps=steps)
X_train_oversample, Y_train_oversample = pipeline.fit_resample(X_train, Y_train.ravel())

# Modeling

**Đoán với data được oversampling**

In [ ]:
# Xây dựng model với xgboost
model = XGBClassifier(n_estimators=900,
    max_depth=10,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=np.nan, # Tham số giúp xử lí missing value
    random_state=2020,
    tree_method='gpu_hist') # Tham số giúp train model bằng gpu
model.fit(X_train_oversample, Y_train_oversample)
del X_train_oversample, Y_train_oversample

In [ ]:
# Đoán trên tập validation
predict = model.predict(X_val)

In [ ]:
# Tính độ chính xác với f1_score, accuracy_score, roc_auc_score
print(f1_score(predict, Y_val))
print(roc_auc_score(predict, Y_val))
print(accuracy_score(predict, Y_val))

**Đoán với data không oversampling**

In [ ]:
# Xây dựng model với xgboost
model = XGBClassifier(n_estimators=900,
    max_depth=10,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=np.nan,
    random_state=2020,
    tree_method='gpu_hist') # Tham số giúp train model bằng gpu
model.fit(X_train, Y_train)
del X_train,Y_train

In [ ]:
# Đoán trên tập validation
predict = model.predict(X_val)
del X_val

In [ ]:
# Tính độ chính xác với f1_score, accuracy_score, roc_auc_score
print(f1_score(predict, Y_val))
print(roc_auc_score(predict, Y_val))
print(accuracy_score(predict, Y_val))

Có thể thấy oversampling làm tăng nhẹ độ chính xác f1 score nhưng đồng thời chỉ số accuracy score và ROC bị giảm. Ở đây bài toán sử dụng độ đo ROC làm độ đo chính, ROC được tính dựa vào 2 chỉ số là False Positive Rate (FPN) (còn gọi là "False Alarm Rate" - tỉ lệ báo động nhầm), và  False Negative Rate (FNR) ( còn gọi là "Miss Detection Rate" - tỉ lệ bỏ sót). Bài toán dự đoán gian lận giao dịch nên việc bỏ sót hành vi gian lận nào đó là thật sự nghiệm trọng và gây hậu quả xấu, vì vậy ROC ở đây sẽ cố gắng làm tăng chỉ số FPR, việc làm này đồng thời làm giảm chỉ số bỏ sót các giao dịch gian lận, vì vậy ta sẽ đề cao các phương pháp có chỉ số ROC cao hơn. Vậy qua thử nghiệm thì các phương pháp oversampling, undersampling hay resampling  đều cho kết quả ROC thấp hơn so với khi không dùng. Model không dùng các phương pháp làm cân bằng data đạt kết quả cao trên tập test public cũng như là test private. 

Xóa các cột có lớn hơn hoặc bằng 95% các giá trị null, hay tiền xử lí emaildomain, thông tin cột id_31 cũng làm model có kết quả chính xác hơn khoảng 0.02%

Ngoài ra việc sử dụng PCA để giảm chiều V feaures làm mô hình hoạt động không tốt. Có lẽ V feautures chứa thông tin quan trong bí mất nào đó của Vesta

Link notebook: https://www.kaggle.com/viethoang303/iee-modeling-pca

# Dự đoán và gửi kết quả

In [ ]:
# Xử lý missing value giống như trên
# test_data.fillna(-999)
X_test = preprocessor.transform(test_data)
# del test_data

In [ ]:
# Dự đoán kết quả
test_predict = model.predict_proba(X_test)
del X_test

In [ ]:
# Tạo DataFrame lưu kết quả
submission = pd.DataFrame({
    'TransactionID' : sample_submission.TransactionID,
    'isFraud' : test_predict[:,1]
})
submission.head()

In [ ]:
# Xuất file kết quả
submission.to_csv('submission.csv', index=False)